In [1]:
import pandas as pd


In [4]:
df = pd.read_parquet("data/NF-UNSW-NB15-V2.parquet")

In [5]:
df

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack
0,1305,21,6,1.0,9,1,193,3,24,24,...,0,7240,0,0,0,0,0,331.0,0,Benign
1,1305,21,6,1.0,261,5,469,7,24,24,...,8688,8688,18944,74,0,0,0,230.0,0,Benign
2,1305,21,6,1.0,481,9,750,11,24,24,...,10136,10136,33792,132,0,0,0,229.0,0,Benign
3,1305,21,6,1.0,701,13,1054,15,24,24,...,11584,11584,48640,190,0,0,0,125.0,0,Benign
4,1305,21,6,1.0,1031,19,1474,21,24,24,...,14480,13032,64256,251,0,0,0,230.0,0,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1986740,20890,5190,6,0.0,1064,12,2260,12,27,27,...,10136,10136,29696,116,0,0,0,0.0,0,Benign
1986741,58663,5190,6,0.0,1064,12,2364,14,27,27,...,10136,10136,29696,116,0,0,0,0.0,0,Benign
1986742,54553,80,6,7.0,994,10,8896,10,26,26,...,10136,7240,7424,29,0,0,0,0.0,0,Benign
1986743,55026,8248,6,0.0,4014,68,60268,70,27,27,...,44888,14480,8960,35,0,0,0,0.0,0,Benign


In [6]:
df.tail()

,L4_SRC_PORT,L4_DST_PORT,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label,Attack
1986740,20890,5190,6,0.0,1064,12,2260,12,27,27,...,10136,10136,29696,116,0,0,0,0.0,0,Benign
1986741,58663,5190,6,0.0,1064,12,2364,14,27,27,...,10136,10136,29696,116,0,0,0,0.0,0,Benign
1986742,54553,80,6,7.0,994,10,8896,10,26,26,...,10136,7240,7424,29,0,0,0,0.0,0,Benign
1986743,55026,8248,6,0.0,4014,68,60268,70,27,27,...,44888,14480,8960,35,0,0,0,0.0,0,Benign
1986744,28987,26948,6,0.0,2246,34,13496,32,26,26,...,20272,14480,8960,35,0,0,0,0.0,0,Benign


In [8]:
df['Attack'].value_counts()

Attack
Benign            1911666
Exploits            29905
Fuzzers             20645
Reconnaissance      11171
Generic              5992
DoS                  4172
Shellcode            1427
Backdoor              833
Analysis              770
Worms                 164
Name: count, dtype: int64